In [33]:
import numpy as np
import cv2 as cv
import glob
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 0.001)
print(cv.TERM_CRITERIA_EPS,cv.TERM_CRITERIA_MAX_ITER,criteria)
a,b=9,6
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((a * b, 3), np.float32)
objp[:, :2] = np.mgrid[0:a, 0:b].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('img/images/*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(img, (a,b), None)
    #print(corners)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (a,b), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
cv.drawChessboardCorners(img, (a,b), corners2, ret)
cv.imshow('img '+str(fname), cv.resize(img,(600,400)))
cv.waitKey(100)
#cv.destroyAllWindows()

2 1 (3, 10, 0.001)


-1

´´´python
    cv.calibrateCamera()
´´´

In [42]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print(ret)

0.28953780687585606


In [43]:
print(mtx)

[[802.32341707   0.         398.06345055]
 [  0.         803.34978321 222.83348372]
 [  0.           0.           1.        ]]


In [44]:
print(dist)

[[ -1.13640344   3.4231306    0.01268819  -0.01488864 -11.1041132 ]]


In [45]:
print(rvecs)

(array([[ 0.12267171],
       [-0.04669564],
       [-0.00990506]]), array([[ 0.12124641],
       [-0.04621453],
       [-0.01057043]]), array([[ 0.12481741],
       [-0.04957144],
       [-0.01033264]]), array([[ 0.12274446],
       [-0.04889983],
       [-0.01107812]]), array([[ 0.12309323],
       [-0.04872764],
       [-0.01097101]]), array([[ 0.12209985],
       [-0.04864226],
       [-0.00987794]]), array([[ 0.12304071],
       [-0.04721454],
       [-0.00894487]]), array([[ 0.12323047],
       [-0.0471175 ],
       [-0.00695773]]), array([[ 0.12412742],
       [-0.04649827],
       [-0.00605563]]), array([[ 0.12614749],
       [-0.04631049],
       [-0.00674124]]), array([[ 0.12734901],
       [-0.04621886],
       [-0.00743291]]), array([[ 0.1266534 ],
       [-0.04589959],
       [-0.0067713 ]]), array([[ 0.12226669],
       [-0.0475654 ],
       [-0.0097812 ]]), array([[ 0.12223756],
       [-0.04876689],
       [-0.00934165]]), array([[ 0.12107872],
       [-0.04842557],
   

In [46]:
print(tvecs)

(array([[-4.87557838],
       [-2.00485797],
       [16.00660884]]), array([[-4.88948401],
       [-1.99449957],
       [16.03761442]]), array([[-4.92398263],
       [-2.03540672],
       [15.88328896]]), array([[-4.93617683],
       [-2.01733968],
       [15.92797273]]), array([[-4.9173663 ],
       [-2.0176874 ],
       [15.94751446]]), array([[-4.8794704 ],
       [-2.01531164],
       [15.94784212]]), array([[-4.84286816],
       [-2.019281  ],
       [15.93530617]]), array([[-4.78627527],
       [-2.02518883],
       [15.91352178]]), array([[-4.74643516],
       [-2.03228747],
       [15.88036184]]), array([[-4.73947705],
       [-2.06269886],
       [15.82528998]]), array([[-4.73604011],
       [-2.07302186],
       [15.80483624]]), array([[-4.72059934],
       [-2.0544958 ],
       [15.82757803]]), array([[-4.88492029],
       [-1.99724808],
       [16.03364889]]), array([[-4.88031837],
       [-1.99551425],
       [16.02602547]]), array([[-4.8674216 ],
       [-1.99612727],
   

In [35]:
img = cv.imread('img/chessboard/patron_1.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [36]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
#dst = dst[y:y+h, x:x+w]
cv.imshow('img '+str(fname), cv.resize(dst,(600,400)))
cv.waitKey(1000)
cv.destroyAllWindows()
cv.imwrite('img/calibresult.png', dst)

True

In [37]:
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# crop the image
x, y, w, h = roi
#dst = dst[y:y+h, x:x+w]
cv.imshow('img '+str(fname), cv.resize(dst,(600,400)))
cv.waitKey(1000)
cv.destroyAllWindows()
cv.imwrite('img/calibresult.png', dst)

True

In [38]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.039389280098404776
